<a href="https://colab.research.google.com/github/victorhmota/cursoml/blob/main/boston_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Definindo uma seed
seed = 10
np.random.seed(seed)

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize = 14)
mpl.rc('xtick', labelsize= 12)
mpl.rc('ytick', labelsize=12)

In [ ]:
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
#Obtendo os dados
from sklearn.datasets import load_boston
housing = load_boston()

In [ ]:
housing

In [ ]:
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.DataFrame(housing.target)

In [ ]:
#Análise exploratória

In [ ]:
X.head() # Perceber que a variável 'CHAS' é uma variável dummy

In [ ]:
X.describe()

In [ ]:
y.describe()

In [ ]:
X.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
#Separando o conjunto treino e teste (proporção 80:20, test_size = 0,20)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = seed)

In [ ]:
#Visualização dos dados
boston = X_train.copy()
boston_prices = y_train.copy()
boston_housing = pd.concat([boston, boston_prices], axis=1)

In [ ]:
corr_matrix = boston_housing.corr()
corr_matrix

In [ ]:
corr_matrix.iloc[:,-1].sort_values(ascending=False) #Alta correlação positiva entre 'RM'(Número de quartos por casa) e o preço. Alta correlação negativa entre população de baixo status e o preço.

In [ ]:
boston_housing.plot(kind='scatter', x = 'RM', y=0, alpha = 0.2)
plt.show()

In [ ]:
boston_housing.plot(kind='scatter', x='LSTAT', y = 0, alpha = 0.2)
plt.show()

In [ ]:
#Verificando existência de dados faltantes
saw_incomplete_rows = boston_housing[boston_housing.isnull().any(axis=1)].head()
saw_incomplete_rows
#Não há dados faltantes

In [ ]:
#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([(('std_scaler'), StandardScaler())])

In [ ]:
boston_prepared = pipeline.fit_transform(boston)

In [ ]:
#Treinando o modelo
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(boston_prepared, boston_prices)

In [ ]:
some_data = boston.iloc[:5]
some_labels = boston_prices.iloc[:5]
some_prepared_data = pipeline.transform(some_data)

lin_reg.predict(some_prepared_data)

In [ ]:
print('Label:',some_labels)

In [ ]:
#Utilizando métricas para avaliar o modelo

from sklearn.metrics import mean_squared_error as MSE
boston_predictons = lin_reg.predict(boston)
lin_mse = MSE(boston_prices, boston_predictons)
lin_rmse = np.sqrt(lin_mse)
print(lin_mse, lin_rmse)


In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

lin_mae = MAE(boston_prices, boston_predictons)
lin_mae

In [ ]:
#Testando outros modelos

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state = seed)
tree_reg.fit(boston_prepared, boston_prices)
tree_prediction = tree_reg.predict(boston_prepared)

In [ ]:
tree_mse = MSE(boston_prices, tree_prediction)
tree_mse

In [ ]:
#Validação cruzada

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, boston_prepared, boston_prices, cv = 10, scoring = 'neg_mean_squared_error')
scores

In [ ]:
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

In [ ]:
def display_scores(scores):
  print('Score:', scores)
  print('Mean:', scores.mean())
  print('SD:', scores.std())

In [ ]:
#Para a Decision Tree
display_scores(tree_rmse_scores)

In [ ]:
#Para a Regressão Linear
lin_rmse_scores = np.sqrt(-cross_val_score(lin_reg, boston_prepared, boston_prices, scoring='neg_mean_squared_error', cv=10))
display_scores(lin_rmse)

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators = 10, random_state=seed)
forest_reg.fit(boston_prepared, boston_prices)

In [ ]:
forest_scores = cross_val_score(forest_reg, boston_prepared, boston_prices, cv=10, scoring='neg_mean_squared_error')
forest_rmse = np.sqrt(-forest_scores)
display_scores(forest_rmse)

In [ ]:
#Aprimorando parâmetros da Floresta Aletória
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators': [3,10,30], 'max_features':[2, 4, 6, 8]}, 
              {'bootstrap': [False], 'n_estimators': [3,10], 'max_features':[2, 3, 4]}]

forest_reg = RandomForestRegressor(random_state=seed)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score = True)
grid_search.fit(boston_prepared, boston_prices)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
  print(np.sqrt(-mean_score), params)

In [ ]:
#Escolhendo o melhor modelo e finalizando

In [ ]:
final_model = grid_search.best_estimator_

X_test_prepared = pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = MSE(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
print(final_mse, final_rmse)

17.76077254901961 4.214353159029225
